# Install all necessary packages, hopefully should work without needing to go into pawsey directories with singularity shell

# MOVE The transformers folder from /src/ in the DNABERT repo to where packages are stored /home/mgill/.local/lib/python3.8/site-package

In [ ]:
#Bug where it installs wrong transformers for some reason in wrong spot, this should fix it

!python3 -m pip install --editable .
!python3 -m pip install torch==1.4 torchvision
!python3 -m pip install tensorflow tensorboardX tensorboard seqeval pyahocorasick scipy statsmodels biopython pandas pybedtools sentencepiece==0.1.91 scikit-learn>=0.22.2
!python3 -m pip install ipywidgets

####MOVE The transformers folder from /src/ in the DNABERT repo to where packages are stored /home/mgill/.local/lib/python3.8/site-package

In [ ]:
!TMPDIR=/home/mgill/ python3 -m pip install scikit-cuda fastai==1.0.61 biopython

In [ ]:
!TMPDIR=/home/mgill/ python3 -m pip install networkx

In [ ]:
!pip list

In [ ]:
from fastai import *
from fastai.text import *
from Bio import Seq
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import FeatureLocation, CompoundLocation
import networkx as nx
import pandas as pd

# Before continuing on ensure you have run script to retrieve fasta from GFF3s (from cantata and ensembl) and random streteches of DNA

### Run Script Below. Needs to have a ref genome file ending in {whatever}_ref.fa
### Needs to have genome info file ending in {whatever}_genome_file.genome
### both need to be in same folder as script

# run bash script below on server with pandas and numpy
# arg1 being url for lncrna, arg2 url for mrna, change URL for each crop

bash create_fasta_from_url.sh http://cantata.amu.edu.pl/DOWNLOADS/Arabidopsis_thaliana_lncrnas.gtf ftp://ftp.ensemblgenomes.org/pub/plants/release-51/gff3/arabidopsis_thaliana/Arabidopsis_thaliana.TAIR10.51.gff3.gz

# Preprocess Data From Fasta of lncrna, mrna and random to TSV's
### Based upon g-ulmfit conversion of fasta to csv
### then DNABERT (with some of my edits) from csv format to kmer tab delimited TSV's

In [ ]:
#set filenames to string
#mrna = 'mrna_ensembl.gff3unique.gff3_flanked.fa'
#lncrna1 = 'lncRNAs_with_gene_cantata.gff3unique.gff3_flanked.fa'
lncrna2 = 'gm_lncrna_flanked.fa'
#randrna1 = 'random_intervals_flanked_w.fa'
randrna2 = 'random_fasta_flanked.fa'

In [ ]:
#read fasta file
#creates DF with column of fasta string and a 2nd column of whether it was mRNA, lncRNA etc.
def parse_fasta(filename, label):
    fasta = SeqIO.parse(filename, 'fasta')
    fs = [i for i in fasta]
    seqs = [str(i.seq) for i in fs]
    df = pd.DataFrame(seqs, columns=['Sequence'])
    df['Target'] = label
    df = df.drop_duplicates()
    return df

In [ ]:
#create df for each set of transcripts
#mrna_df = parse_fasta(mrna, "not_lncRNA")
#lncrna_df1 = parse_fasta(lncrna1, "lncRNA")
lncrna_df2 = parse_fasta(lncrna2, "lncRNA")
#random_df1 = parse_fasta(randrna1, "not_lncRNA")
random_df2 = parse_fasta(randrna2, "not_lncRNA")

In [ ]:
#mrna_df=mrna_df.sample(1000)
#mrna_df.shape
#print(mrna_df)

In [ ]:
#random_df=random_df.sample(4000)
#random_df.shape
#print(random_df)

In [ ]:
total = 0
for x in range(0,random_df2.shape[0]):
    y=len(random_df2.iloc[x]["Sequence"])
    total=total+y
print(total)
    
print(str(total/random_df2.shape[0]))

total = 0
for x in range(0,lncrna_df2.shape[0]):
    y=len(lncrna_df2.iloc[x]["Sequence"])
    if(y>total):
        total=y
print(total)
    
print(str(total/lncrna_df2.shape[0]))

In [ ]:
#divide data into a holdout set and training test set, hardcoded at 80/20 split
def partition_data(df):
    
    train_size = int(len(df)*0.80)
    ho_size = int(len(df)) - train_size
    
    train_df = df.sample(train_size)
    ho_df = df.drop(train_df.index)
    #final column of what it was set to 
    train_df['set'] = 'train'
    ho_df['set'] = 'holdout'
    
    return train_df, ho_df

In [ ]:
#split data into training and holdout
#train_mrna_df, ho_mrna_df = partition_data(mrna_df)
#train_lncrna_df, ho_lncrna_df = partition_data(lncrna_df1)
train_lncrna_df2, ho_lncrna_df2 = partition_data(lncrna_df2)
#train_random_df1, ho_random_df1 = partition_data(random_df1)
train_random_df2, ho_random_df2 = partition_data(random_df2)

In [ ]:
#concat all training data together and holdout data together
train_test_df = pd.concat([train_lncrna_df2, train_random_df2], axis = 0, ignore_index = True)
holdout_df = pd.concat([ho_lncrna_df2, ho_random_df2], axis = 0, ignore_index = True)

In [ ]:
#drop set column
train_test_df.drop("set", axis = 1, inplace = True)
holdout_df.drop("set", axis = 1, inplace = True)

forbidden_strings = ['N', 'Y', 'D', 'S', 'K', 'W', 'M', 'R', 'V', 'H', 'B']

#check shape before removal
print(train_test_df.shape)
print(holdout_df.shape)

#Remove any rows with forbidden strings in sequence by equalling rows that don't (~) have 'forbidden string' in sequence
for S in forbidden_strings:
    train_test_df = train_test_df[~train_test_df.Sequence.map(lambda x: S in x.upper())]
    holdout_df = holdout_df[~holdout_df.Sequence.map(lambda x: S in x.upper())]

#check shape after removal
print(train_test_df.shape)
print(holdout_df.shape)

In [ ]:
#shuffle
train_test_df = train_test_df.sample(frac=1)
print(train_test_df)
holdout_df = holdout_df.sample(frac=1)
print(holdout_df)

#save to file
train_test_df.to_csv("train_test.csv", index=False)
holdout_df.to_csv("holdout.csv", index=False)

In [ ]:
#Unccoment first time you run this cell
#!mv motif/motif_utils.py .
from motif_utils import *
import sys
def csv_to_dnabert_tsv(in_string, out_string, kmer):
    csv_file = in_string
    my_k = kmer
    df = pd.read_csv(csv_file)
    seq_array = []
    label_array = []
    for i, row in df.iterrows():
        seq = row["Sequence"]
        label = row["Target"]
        seq = seq2kmer(seq,my_k)
        seq_array.append(seq)
        if(label == "lncRNA"):
               label = 0
        else:
               label = 1
        label_array.append(label)
    data = np.array([seq_array, label_array])
    data = data.T
    print(data)
    new_df = pd.DataFrame(data, columns=["sequence", "label"], index=None)
    print(new_df)
    new_df.to_csv(out_string, sep="\t", header=True, index=False)

In [ ]:
#Should now be in format that is DNABERT friendly, output has to be train.tsv and dev.tsv for DNABERT
#final arg is kmer size, adjust for which model you've downloaded from their pretrained
csv_to_dnabert_tsv("train_test.csv", "train.tsv", 3)
csv_to_dnabert_tsv("holdout.csv", "dev.tsv", 3)
!mv *.tsv input_data_3mer/

csv_to_dnabert_tsv("train_test.csv", "train.tsv", 4)
csv_to_dnabert_tsv("holdout.csv", "dev.tsv", 4)
!mv *.tsv input_data_4mer/

csv_to_dnabert_tsv("train_test.csv", "train.tsv", 5)
csv_to_dnabert_tsv("holdout.csv", "dev.tsv", 5)
!mv *.tsv input_data_5mer/

csv_to_dnabert_tsv("train_test.csv", "train.tsv", 6)
csv_to_dnabert_tsv("holdout.csv", "dev.tsv", 6)
!mv *.tsv input_data_6mer/

# Fine Tune Models with Arabidopsis Data

## 3mer

In [ ]:
#torch.cuda.empty_cache()
#hardocded the variables, not sure how to do this with env variables above :(
#!mv examples/run_finetune.py .
%run run_finetune.py \
    --model_type dnalongcat \
    --tokenizer_name=dna3 \
    --model_name_or_path=/group/pawsey0149/mgill/dl_playground/DNABERT_Gm/3-new-12w-0/ \
    --task_name dnaprom \
    --do_train \
    --do_eval \
    --data_dir /group/pawsey0149/mgill/dl_playground/DNABERT_Gm/input_data_3mer \
    --max_seq_length 2048 \
    --per_gpu_eval_batch_size=4   \
    --per_gpu_train_batch_size=4   \
    --learning_rate 2e-5 \
    --num_train_epochs 4.0 \
    --output_dir /group/pawsey0149/mgill/dl_playground/DNABERT_Gm/output_data_3mer/ \
    --evaluate_during_training \
    --logging_steps 100 \
    --save_steps 4000 \
    --warmup_percent 0.1 \
    --hidden_dropout_prob 0.1 \
    --overwrite_output \
    --weight_decay 0.01 \
    --n_process 8

In [ ]:
%run run_finetune.py \
    --model_type dnalongcat \
    --tokenizer_name=dna3 \
    --model_name_or_path /group/pawsey0149/mgill/dl_playground/DNABERT_Gm/output_data_3mer/ \
    --task_name dnaprom \
    --do_predict \
    --data_dir /group/pawsey0149/mgill/dl_playground/DNABERT_Gm/input_data_3mer \
    --max_seq_length 2048 \
    --per_gpu_pred_batch_size=8   \
    --output_dir /group/pawsey0149/mgill/dl_playground/DNABERT_Gm/output_data_3mer/ \
    --predict_dir /group/pawsey0149/mgill/dl_playground/DNABERT_Gm/results_3mer/

## 4mer

In [ ]:
#hardocded the variables, not sure how to do this with env variables above :(
#!mv examples/run_finetune.py .
%run run_finetune.py \
    --model_type dnalongcat \
    --tokenizer_name=dna4 \
    --model_name_or_path=/group/pawsey0149/mgill/dl_playground/DNABERT_Gm/4-new-12w-0/ \
    --task_name dnaprom \
    --do_train \
    --do_eval \
    --data_dir /group/pawsey0149/mgill/dl_playground/DNABERT_Gm/input_data_4mer/ \
    --max_seq_length 2048 \
    --per_gpu_eval_batch_size=4   \
    --per_gpu_train_batch_size=4   \
    --learning_rate 2e-5 \
    --num_train_epochs 4.0 \
    --output_dir /group/pawsey0149/mgill/dl_playground/DNABERT_Gm/output_data_4mer/ \
    --evaluate_during_training \
    --logging_steps 100 \
    --save_steps 4000 \
    --warmup_percent 0.1 \
    --hidden_dropout_prob 0.1 \
    --overwrite_output \
    --weight_decay 0.01 \
    --n_process 8

In [ ]:
%run run_finetune.py \
    --model_type dnalongcat \
    --tokenizer_name=dna4 \
    --model_name_or_path /group/pawsey0149/mgill/dl_playground/DNABERT_Gm/output_data_4mer/ \
    --task_name dnaprom \
    --do_predict \
    --data_dir /group/pawsey0149/mgill/dl_playground/DNABERT_Gm/input_data_4mer \
    --max_seq_length 2048 \
    --per_gpu_pred_batch_size=8   \
    --output_dir /group/pawsey0149/mgill/dl_playground/DNABERT_Gm/output_data_4mer/ \
    --predict_dir /group/pawsey0149/mgill/dl_playground/DNABERT_Gm/results_4mer/

## 5mer

In [ ]:
#hardocded the variables, not sure how to do this with env variables above :(
#!mv examples/run_finetune.py .
%run run_finetune.py \
    --model_type dnalongcat \
    --tokenizer_name=dna5 \
    --model_name_or_path=/group/pawsey0149/mgill/dl_playground/DNABERT_Gm/5-new-12w-0/ \
    --task_name dnaprom \
    --do_train \
    --do_eval \
    --data_dir /group/pawsey0149/mgill/dl_playground/DNABERT_Gm/input_data_5mer/ \
    --max_seq_length 2048 \
    --per_gpu_eval_batch_size=4   \
    --per_gpu_train_batch_size=4   \
    --learning_rate 2e-5 \
    --num_train_epochs 4.0 \
    --output_dir /group/pawsey0149/mgill/dl_playground/DNABERT_Gm/output_data_5mer/ \
    --evaluate_during_training \
    --logging_steps 100 \
    --save_steps 4000 \
    --warmup_percent 0.1 \
    --hidden_dropout_prob 0.1 \
    --overwrite_output \
    --weight_decay 0.01 \
    --n_process 8

In [ ]:
%run run_finetune.py \
    --model_type dnalongcat \
    --tokenizer_name=dna5 \
    --model_name_or_path /group/pawsey0149/mgill/dl_playground/DNABERT_Gm/output_data_5mer/ \
    --task_name dnaprom \
    --do_predict \
    --data_dir /group/pawsey0149/mgill/dl_playground/DNABERT_Gm/input_data_5mer \
    --max_seq_length 2048 \
    --per_gpu_pred_batch_size=8   \
    --output_dir /group/pawsey0149/mgill/dl_playground/DNABERT_Gm/output_data_5mer/ \
    --predict_dir /group/pawsey0149/mgill/dl_playground/DNABERT_Gm/results_5mer/

## 6mer

In [ ]:
#hardocded the variables, not sure how to do this with env variables above :(
#!mv examples/run_finetune.py .
%run run_finetune.py \
    --model_type dnalongcat \
    --tokenizer_name=dna6 \
    --model_name_or_path=/group/pawsey0149/mgill/dl_playground/DNABERT_Gm/6-new-12w-0/ \
    --task_name dnaprom \
    --do_train \
    --do_eval \
    --data_dir /group/pawsey0149/mgill/dl_playground/DNABERT_Gm/input_data_6mer/ \
    --max_seq_length 2048 \
    --per_gpu_eval_batch_size=2   \
    --per_gpu_train_batch_size=2   \
    --learning_rate 2e-5 \
    --num_train_epochs 4.0 \
    --output_dir /group/pawsey0149/mgill/dl_playground/DNABERT_Gm/output_data_6mer/ \
    --evaluate_during_training \
    --logging_steps 100 \
    --save_steps 4000 \
    --warmup_percent 0.1 \
    --hidden_dropout_prob 0.1 \
    --overwrite_output \
    --weight_decay 0.01 \
    --n_process 8

In [ ]:
%run run_finetune.py \
    --model_type dnalongcat \
    --tokenizer_name=dna6 \
    --model_name_or_path /group/pawsey0149/mgill/dl_playground/DNABERT_Gm/output_data_6mer/ \
    --task_name dnaprom \
    --do_predict \
    --data_dir /group/pawsey0149/mgill/dl_playground/DNABERT_Gm/input_data_6mer \
    --max_seq_length 2048 \
    --per_gpu_pred_batch_size=8   \
    --output_dir /group/pawsey0149/mgill/dl_playground/DNABERT_Gm/output_data_6mer/ \
    --predict_dir /group/pawsey0149/mgill/dl_playground/DNABERT_Gm/results_6mer/

# DNABERT Motif Analysis

In [ ]:
#Run the attention score generation
%run run_finetune.py \
    --model_type dnalongcat \
    --tokenizer_name=dna3 \
    --model_name_or_path /group/pawsey0149/mgill/dl_playground/DNABERT_Gm/output_data_3mer/ \
    --task_name dnaprom \
    --do_visualize \
    --visualize_data_dir /group/pawsey0149/mgill/dl_playground/DNABERT_Gm/input_data_3mer/ \
    --data_dir /group/pawsey0149/mgill/dl_playground/DNABERT_Gm/input_data_3mer/ \
    --max_seq_length 2048 \
    --per_gpu_pred_batch_size=8   \
    --output_dir /group/pawsey0149/mgill/dl_playground/DNABERT_Gm/output_data_3mer/ \
    --predict_dir /group/pawsey0149/mgill/dl_playground/DNABERT_Gm/results_3mer/

In [ ]:
#test attention scores have been loaded
import numpy as np
my_at = np.load("/group/pawsey0149/mgill/dl_playground/DNABERT_Gm/results_3mer/atten.npy")
print(my_at.shape)

In [ ]:
#Visualise
%run find_motifs.py \
    --data_dir /group/pawsey0149/mgill/dl_playground/DNABERT_Gm/input_data_3mer \
    --predict_dir /group/pawsey0149/mgill/dl_playground/DNABERT_Gm/results_3mer/ \
    --window_size 24 \
    --min_len 5 \
    --pval_cutoff 0.05 \
    --min_n_motif 3 \
    --align_all_ties \
    --save_file_dir /group/pawsey0149/mgill/dl_playground/DNABERT_Gm/results_3mer/ \
    --verbose

# Transformers Interpret Motif Finder

In [1]:
import torch
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import BertForLongSequenceClassification, BertForSequenceClassification
from transformers_interpret import SequenceClassificationExplainer

#load variables for the classification explainer
config = AutoConfig.from_pretrained("output_data_3mer/", output_attentions=True)
tokenizer = AutoTokenizer.from_pretrained("output_data_3mer/")
#have to use custom LONG bert model
model = BertForLongSequenceClassification.from_pretrained("output_data_3mer/", config=config)

#load classification explainer
cls_explainer = SequenceClassificationExplainer(
    model,
    tokenizer)

<class 'transformers.tokenization_bert.BertTokenizerFast'>


In [2]:
import pandas as pd
import numpy as np

#load the holdout data
holdout = pd.read_csv("input_data_3mer/dev.tsv", sep = "\t")

#finds the indexes that were predicted to be 0 (aka lncRNA)
#and matches it with the labels that were ACTUALLY 0
#returns indexes of these matches to minimise cls explainers to go through
pred_results = np.load("results_3mer/pred_results.npy")
my_df = pd.read_csv("input_data_3mer/dev.tsv", sep = "\t")
#print(my_df)
targets = np.array(my_df["label"])
rounded_preds = [int(round(value)) for value in pred_results]
lncrna_idx = []
count = 0
print(len(targets))
print(len(rounded_preds))
while count < len(targets):
    #if target matches prediction and the prediction is for a lncrna store count in lncrna_idx
    if((targets[count] == 0)):
      #  print(count)
        lncrna_idx.append(count)
    count = count + 1
print((len(lncrna_idx)))

874
874
436


In [3]:
#inputs a set of word attributions from cls explainer and the current motif dictionary
#returns and updated motif dictionary
#mc in a dictionary with the motif and the number of times it appears
def make_motif_dict(word_attributions, motif_coll, mc):
    motif = ''
    motif_val = 0
    for x in range(len(word_attributions)):
        y = word_attributions[x]
        current_string = y[0]
        if(current_string != '[PAD]' and current_string != '[CLS]' and current_string != '[SEP]'):
            value = y[1]
            if(value>0):
                if(motif==''):
                    motif=current_string
                    motif_val = value
                else:
                    #change this value to be the length of kmer - 1 so that way we just adding the new base
                    motif=motif+current_string[2]
                    motif_val = motif_val + value
            else:
                if(len(motif)<7): ##makes sure its more than just a codon
                    motif = ''
                if(motif != ''):
                    #print('finished motif')
                    #print(motif)
                    #print(motif_val)
                    if motif in motif_coll:
                        print('we found a double boys yeehaw')
                        old_val = motif_coll.get(motif)
                        new_val = motif_val + old_val
                       # print(old_val)
                       # print(new_val)
                        #gets current count for motif, adds 1 and updates the motif_count dict
                        mc_old = mc.get(motif)
                        mc_new = mc_old + 1
                        motif_coll.update({motif:new_val})
                        print("motif: "+str(motif) + "   count: " + str(mc_new))
                        mc.update({motif:mc_new})
                    else:
                        mc[motif] = 1
                        motif_coll[motif] = motif_val
                        motif = ''
    return motif_coll, mc

In [4]:
#goes through each sequence that is lncRNA and is predicted be lncRNA from model

#goes through each sequence that is lncRNA and is predicted be lncRNA from model

m = {}
m_count = {}
for i in (lncrna_idx):
   # if(i>100):
   #     break
    example = holdout['sequence'][i]
    print(i)
    word_attributions = cls_explainer(example)
    print("predicted index from cls: " + str(cls_explainer.predicted_class_index))
    if(cls_explainer.predicted_class_index == 0):
        m,m_count = make_motif_dict(word_attributions, m, m_count)


0
this is embeddings: Embedding(69, 768, padding_idx=0)
this is the indexes: None
this is the class name: None
7
2048
ref id length is: 2048
this is self input ids before preds call: tensor([[ 2, 22, 10,  ..., 21,  8,  3]])
this is input ids before ligattributionstensor([[ 2, 22, 10,  ..., 21,  8,  3]])
this is input ids shape before preds call: torch.Size([1, 2048])
this is position shape: torch.Size([1, 2048])
new pos ids = tensor([[  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511]])
this is AM: tensor([1., 1., 1.,  ..., 1., 1., 1.])
this is input ids shape before preds call: torch.Size([1, 2048])
this is position shape: torch.Size([1, 2048])
new pos ids = tensor([[  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511]])
this is AM: tensor([1., 

predicted index from cls: 0
this is self input ids before preds call: tensor([[ 2, 54, 11,  ...,  0,  0,  0]])
6
this is embeddings: Embedding(69, 768, padding_idx=0)
this is the indexes: None
this is the class name: None
17
2048
ref id length is: 2048
this is self input ids before preds call: tensor([[ 2, 22, 10,  ..., 22, 10,  3]])
this is input ids before ligattributionstensor([[ 2, 22, 10,  ..., 22, 10,  3]])
this is input ids shape before preds call: torch.Size([1, 2048])
this is position shape: torch.Size([1, 2048])
new pos ids = tensor([[  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511]])
this is AM: tensor([1., 1., 1.,  ..., 1., 1., 1.])
this is input ids shape before preds call: torch.Size([1, 2048])
this is position shape: torch.Size([1, 2048])
new pos ids = tensor([[  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511],
     

this is self input ids before preds call: tensor([[ 2, 22, 11,  ...,  0,  0,  0]])
predicted index from cls: 0
this is self input ids before preds call: tensor([[ 2, 22, 11,  ...,  0,  0,  0]])
12
this is embeddings: Embedding(69, 768, padding_idx=0)
this is the indexes: None
this is the class name: None
2
2048
ref id length is: 2048
this is self input ids before preds call: tensor([[ 2, 28, 34,  ...,  0,  0,  0]])
this is input ids before ligattributionstensor([[ 2, 28, 34,  ...,  0,  0,  0]])
this is input ids shape before preds call: torch.Size([1, 2048])
this is position shape: torch.Size([1, 2048])
new pos ids = tensor([[  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511]])
this is AM: tensor([1., 1., 1.,  ..., 0., 0., 0.])
this is input ids shape before preds call: torch.Size([1, 2048])
this is position shape: torch.Size([1, 2048])
new pos ids = tensor([[  0, 

this is self input ids before preds call: tensor([[ 2, 33, 55,  ..., 58, 25,  3]])
predicted index from cls: 0
this is self input ids before preds call: tensor([[ 2, 33, 55,  ..., 58, 25,  3]])
16
this is embeddings: Embedding(69, 768, padding_idx=0)
this is the indexes: None
this is the class name: None
4
2048
ref id length is: 2048
this is self input ids before preds call: tensor([[ 2, 43, 32,  ..., 63, 48,  3]])
this is input ids before ligattributionstensor([[ 2, 43, 32,  ..., 63, 48,  3]])
this is input ids shape before preds call: torch.Size([1, 2048])
this is position shape: torch.Size([1, 2048])
new pos ids = tensor([[  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511]])
this is AM: tensor([1., 1., 1.,  ..., 1., 1., 1.])
this is input ids shape before preds call: torch.Size([1, 2048])
this is position shape: torch.Size([1, 2048])
new pos ids = tensor([[  0, 

this is input ids shape before preds call: torch.Size([4, 512])
this is position shape: torch.Size([4, 512])
this is AM: tensor([1., 1., 1.,  ..., 1., 1., 1.])
this is self input ids before preds call: tensor([[ 2,  6, 12,  ..., 22,  9,  3]])
predicted index from cls: 0
this is self input ids before preds call: tensor([[ 2,  6, 12,  ..., 22,  9,  3]])
20
this is embeddings: Embedding(69, 768, padding_idx=0)
this is the indexes: None
this is the class name: None
3
2048
ref id length is: 2048
this is self input ids before preds call: tensor([[ 2, 61, 38,  ...,  0,  0,  0]])
this is input ids before ligattributionstensor([[ 2, 61, 38,  ...,  0,  0,  0]])
this is input ids shape before preds call: torch.Size([1, 2048])
this is position shape: torch.Size([1, 2048])
new pos ids = tensor([[  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511]])
this is AM: tensor([1., 1., 1.

...

this is input ids before ligattributionstensor([[ 2, 36, 65,  ...,  0,  0,  0]])
this is input ids shape before preds call: torch.Size([1, 2048])
this is position shape: torch.Size([1, 2048])
new pos ids = tensor([[  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511]])
this is AM: tensor([1., 1., 1.,  ..., 0., 0., 0.])
this is input ids shape before preds call: torch.Size([1, 2048])
this is position shape: torch.Size([1, 2048])
new pos ids = tensor([[  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511]])
this is AM: tensor([1., 1., 1.,  ..., 0., 0., 0.])
this is input ids shape before preds call: torch.Size([50, 2048])
this is position shape: torch.Size([50, 2048])
new pos ids = tensor([[  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2, 

(tensor([[ 2, 12, 36,  ..., 21,  6,  3]]), tensor([[   0,    1,    2,  ..., 2045, 2046, 2047]]))
(tensor([[2, 0, 0,  ..., 0, 0, 3]]), tensor([[0, 0, 0,  ..., 0, 0, 0]]))
0
[tensor([[ 2, 12, 36,  ..., 21,  6,  3],
        [ 2,  9, 22,  ..., 43, 30,  3],
        [ 2, 43, 29,  ..., 39, 14,  3],
        [ 2, 42, 25,  ..., 21,  6,  3]]), tensor([[  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511]])]
torch.Size([4, 512])
torch.Size([4, 512])
this is input ids shape before preds call: torch.Size([1, 2048])
this is position shape: torch.Size([1, 2048])
new pos ids = tensor([[  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511]])
this is AM: tensor([1., 1., 1.,  ..., 1., 1., 1.])
this is input ids shape before preds call: torch.Size([4, 512])
this is posi

this is input ids before ligattributionstensor([[2, 6, 9,  ..., 0, 0, 0]])
this is input ids shape before preds call: torch.Size([1, 2048])
this is position shape: torch.Size([1, 2048])
new pos ids = tensor([[  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511]])
this is AM: tensor([1., 1., 1.,  ..., 0., 0., 0.])
this is input ids shape before preds call: torch.Size([1, 2048])
this is position shape: torch.Size([1, 2048])
new pos ids = tensor([[  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511]])
this is AM: tensor([1., 1., 1.,  ..., 0., 0., 0.])
this is input ids shape before preds call: torch.Size([50, 2048])
this is position shape: torch.Size([50, 2048])
new pos ids = tensor([[  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 

this is input ids before ligattributionstensor([[ 2,  9, 24,  ...,  0,  0,  0]])
this is input ids shape before preds call: torch.Size([1, 2048])
this is position shape: torch.Size([1, 2048])
new pos ids = tensor([[  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511]])
this is AM: tensor([1., 1., 1.,  ..., 0., 0., 0.])
this is input ids shape before preds call: torch.Size([1, 2048])
this is position shape: torch.Size([1, 2048])
new pos ids = tensor([[  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511]])
this is AM: tensor([1., 1., 1.,  ..., 0., 0., 0.])
this is input ids shape before preds call: torch.Size([50, 2048])
this is position shape: torch.Size([50, 2048])
new pos ids = tensor([[  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2, 

this is input ids before ligattributionstensor([[ 2,  9, 22,  ..., 61, 40,  3]])
this is input ids shape before preds call: torch.Size([1, 2048])
this is position shape: torch.Size([1, 2048])
new pos ids = tensor([[  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511]])
this is AM: tensor([1., 1., 1.,  ..., 1., 1., 1.])
this is input ids shape before preds call: torch.Size([1, 2048])
this is position shape: torch.Size([1, 2048])
new pos ids = tensor([[  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2,  ..., 509, 510, 511]])
this is AM: tensor([1., 1., 1.,  ..., 1., 1., 1.])
this is input ids shape before preds call: torch.Size([50, 2048])
this is position shape: torch.Size([50, 2048])
new pos ids = tensor([[  0,   1,   2,  ..., 509, 510, 511],
        [  0,   1,   2, 

In [5]:
#iterate through the dictionary
final_motifs = {}
for x in m:
    att_val = m.get(x)
    count = m_count.get(x)
    val = float(att_val*count)
    print(x)
    print(val)
    final_motifs[x] = val
print(final_motifs)

CTTGTGC
0.005492084133901704
CACAGGGA
0.039950614232824394
TTCCCTTA
0.012884343179333926
ACAGCCTCCA
0.014126261018379348
TGAGGTT
7.712599724494357
TTCAAGA
0.008535279016553207
CTGTTGTTTG
0.04532110623854506
TCCCTCTATG
0.17150772861872587
TGAGTCA
0.08237233075151082
GTTGTTA
0.4090769122972495
TTTCATTTTA
0.017224838259072076
GGTAATA
0.013588717034828154
AATAATA
181.8098898150852
TTTTATAATA
0.045120249264411626
TCTGATTA
0.04633131854761603
GTACATGC
0.09724481006917923
CTATTGTATGGT
0.2830784500743422
TTTTAATAAAAAAAA
0.1270767841419401
TGGAGTT
0.12269603078211344
AAATTTCTGT
0.07436776851341897
AACTGGT
0.12629367280671738
TCCATGT
0.13940452321431637
TGGACAG
0.025603264842032784
TGGTGGG
0.874975857766316
TTACATAAT
0.07467076587444792
TTAGAAAT
0.0490423369204965
ACGAATACCGA
0.09418496685989229
AATTTAA
35.51632048842999
GTTTGTCACAAT
0.12574246205238093
GCAAATT
1.3322629204934606
TGATACC
0.27555955465731125
TTAACAAA
0.3398919520936248
GTTATAA
21.821495489553875
TTTTTAA
17.83759601446497
GATGTAAA

0.15223150805839003
AAAAAGGA
0.03223862451542984
ACCACTGA
0.06860734571459856
GCCTTAAT
0.07649735113857412
TGTATTTCG
0.12363478702091137
GTAATTT
0.11995131316031238
ACATTAAA
2.664269877541996
ACATTTT
0.8375080120626626
TAAAATAG
0.06198712268101774
AATTATA
0.9187054179053937
TTGTAGA
0.050122224680336745
AAAATAC
1.5397812044884878
CAATTTTAA
0.057703295928491126
AATTGGT
0.7654766771829766
CTGTTGTTAAT
0.4751493660024693
TATGTTAA
1.1140815965634876
TCGTAGC
0.1332212470204603
TTGTTTTT
0.038708134677240574
CAGGGTTTGA
0.016542611443994847
AAAACGTGGCG
0.10763591638539954
GCAGGAG
1.7203545576715262
GAGAGGACGAGGT
0.03728376638148985
TGATGGT
1.5439551298597631
GCGGTGGGA
0.1677089070009078
CGCTGGTGATC
0.10315883116741231
CCATGGGCCAGCCCGGTACACT
0.20977816009407343
TGGAAATCAGA
0.07409167241892464
TCAATTTTTT
0.12444099804913573
TTAAGAAAATAATACCAT
0.12361967911112252
AAGTTTT
0.01778732600920065
CCACTTAATATG
0.078978042349273
CATATTTAA
0.019149400548232554
AATATTTTG
0.023879595990929748
TTAATTTTTTTTAA
0

ATATGTAGT
0.05368493395544252
ACCCCACATTATATATA
0.16269550650485526
GCGGCGA
0.0456905052515627
GCACCAT
0.0247421838304678
TTATTTTACT
0.23723473228500366
TTTATATT
7.718191069474307
TTATCTACAAC
0.5200660389447492
TAAAAATATTAAT
0.38129928906124966
TTACGAG
0.10350944801211792
CTTATATTTTCC
0.1934063311539176
GTGCATG
0.4576412136939525
TGCTATTATT
0.10047446260241079
TTATTAGATATC
0.09959117930552223
TTGATAAAAA
0.12797238925448154
ATGCTAAT
0.11699756305563251
TACGTACATGTAA
0.17370275202311472
TTTTTTCCT
0.0761323365019239
TAGATCG
0.0920347365364826
AAGTTTAA
0.07321713726913694
TTATAAAAAG
0.12389077449532551
TATGTTT
1.8931332741632743
AATCAAA
0.1269919236679524
AGCCTCCACCA
0.28918785396369684
ACTGATG
0.08350923749474311
TCTCCAC
0.039441552857678475
TCTCCCT
0.18200630681543328
TCCAGCAGCCGG
0.33988640495465894
CCAGAGGCA
0.059607351993943555
CAGCAGTTTGTTAGGGT
0.2470243521666367
CACAGATGTCGAAG
0.7156901027128113
TTGTGGACA
0.23641923910893387
GCCCACACA
0.3593335593153415
CTTTAATTTTT
0.094413739014064

In [6]:
import json

json = json.dumps(final_motifs)
print(json)
f = open("Gm_motif_dict_idx.json","w")
f.write(json)
f.close()

In [7]:
import json
ff = open("Gm_motif_dict_idx.json")
final_motifs = json.load(ff)

from PIL import Image
import matplotlib.pyplot as plt
from wordcloud import WordCloud

wc = WordCloud(background_color="white",width=1000,height=1000, max_words=10,relative_scaling=0.5,normalize_plurals=False).generate_from_frequencies(final_motifs)
plt.imshow(wc)

In [8]:
#tests the json file opens
import json
ff = open("Gm_motif_dict_idx.json")
final_motifs = json.load(ff)


#prints highest value motifs 
import operator
sorted_m = sorted(final_motifs.items(), key=lambda kv: kv[1])
#sorted_m.reverse() = sorted(m.items(), key=operator.itemgetter(1))
sorted_m.reverse()
sorted_m

In [9]:
top_50 = []
for x in sorted_m:
    top_50.append(x[0])
    if(len(top_50) == 50):
        break

In [10]:
with open("Gm_motif_top50.txt", "w") as txt_file:
    i = 0
    for line in top_50:
        if(i<49):
            txt_file.write(line + "\n")
            i = i + 1
        else:
            txt_file.write(line)

In [ ]:
### send this text file to the bash / R combination of significance
### copy back the motifs and sigs.txt

In [11]:
lines = []
sigs = open("motif_sigs.txt")
for line in sigs:
    line = line.strip()
    if(" " in line):
        line = line.split()
        line = float(line[1])
    lines.append(line)
print(lines)

In [12]:
top_50_sig_motifs = {}
i = 0
while i < len(lines):
    my_motif = lines[i]
    sig_value = lines[i+1]
    if(sig_value<0.05):
        top_50_sig_motifs[my_motif] = sig_value
    i = i + 2
print(len(top_50_sig_motifs))
print(top_50_sig_motifs)

In [13]:
#store top 50 motifs that are sig into new json file
import json

json = json.dumps(top_50_sig_motifs)
print(json)
f = open("Gm_sig_motifs_top50.json","w")
f.write(json)
f.close()

# Classifier Based off Presence of Sig Motifs

In [ ]:
#load motifs that were significant in top 50 attention score motifs
import json
ff = open("Gm_sig_motifs_top50.json")
sig_motifs_in_top50 = json.load(ff)
ff.close()

In [ ]:
#load sequences and equivalent label

import pandas as pd
df = pd.read_csv("holdout.csv")
print(df)
test_seqs = df["Sequence"]
test_labels=[]
for x in df["Target"]:
    if("not_lncRNA" in x):
        test_labels.append(1)
    else:
        test_labels.append(0)

In [ ]:
from sklearn.metrics import accuracy_score

accuracies = []

#loop through different thresholds to find best threshold of accuracy
for i in range(0,len(sig_motifs_in_top50)):
    #loop through each available sequences to test
    pred_labels = []
    for seq in test_seqs:
        #how many of the motifs are found in the seq
        sig_motif_count = 0
        for motif in sig_motifs_in_top50:
            if motif in seq:
                sig_motif_count = sig_motif_count + 1
        #check to see if the # of motifs found is above the current test threshold
        if(sig_motif_count >= i):
            pred_labels.append(0)
        else:
            pred_labels.append(1)
    accuracy = accuracy_score(test_labels, pred_labels)
    accuracies.append(accuracy)
    print("Accuracy for threshold "+str(i)+" is "+str(accuracy))

In [ ]:
from sklearn.metrics import confusion_matrix

pred_labels = []
for seq in test_seqs:
    #how many of the motifs are found in the seq
    sig_motif_count = 0
    for motif in sig_motifs_in_top50:
        if motif in seq:
            sig_motif_count = sig_motif_count + 1
    #check to see if the # of motifs found is above the current test threshold
    if(sig_motif_count >= 7):
        pred_labels.append(0)
    else:
        pred_labels.append(1)
accuracy = accuracy_score(test_labels, pred_labels)

cm = confusion_matrix(test_labels, pred_labels)
print(cm)